# Projet 2 : Analyse des données de systèmes éducatifs

## 0. Importations des modules et des données

In [796]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline
#% matplotlib notebook
# graphes interactifs
import re
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
#import scipy.stats as st
#import statsmodels.api as sm
#from sklearn.datasets import load_iris
#iris_df_ori = load_iris()

####     A ESSAYER      #######################################################
# GRAPHES INTERACTIFS


# POUR LES GRAPHIQUES
# % matplotlib inline 
# plt.rcParams['figure.figsize'] = [9.5, 6] # ajuster la taille

# POUR DESACTIVER LA TOOLBOX GRAPHES TOP GRANDS
# %%javascript
# IPython.OutputArea.prototype._should_scroll = function(lines) {
#     return false;
# }


L'ensemble des données téléchargées se compose de 5 fichiers .csv et d'un fichier excel comportant 5 onglets.
Il semble que l'intégralité des données des fichiers .csv soit reprise dans chacun des onglets du fichier Excel.

Dans ce notebook, on appellera "base de donnée" l'ensemble des données, et "table" chacun des onglets ou fichier .csv correspondant.

In [ ]:
# Utilisé la fonction dropna (colonne nulle) pour éliminer les colonnes fantômes "Unamed en fin de tableau"
data = pd.read_csv("../DONNEES/EdStatsData.csv").dropna(how='all', axis='columns')
country = pd.read_csv("../DONNEES/EdStatsCountry.csv").dropna(how='all', axis='columns')
cnt_ser = pd.read_csv("../DONNEES/EdStatsCountry-Series.csv").dropna(how='all', axis='columns')
series = pd.read_csv("../DONNEES/EdStatsSeries.csv").dropna(how='all', axis='columns')
footnote = pd.read_csv("../DONNEES/EdStatsFootNote.csv").dropna(how='all', axis='columns')

## 1. Description globale des 5 tables

In [ ]:
## Fonction qui extrait les données principales relatives aux colonnes

def infos_df (df):
    infos = pd.DataFrame(df.dtypes).T.rename(index={0:'Type'}) 
    infos = infos.append(pd.DataFrame([df[col].size for col in df.columns],index=df.columns)\
                               .T.rename(index={0:'Nbe lignes'}))
    infos = infos.append(pd.DataFrame(df.isna().sum()).T.rename(index={0:'Nbe NaN'}))
    infos = infos.append(pd.DataFrame(infos.loc["Nbe lignes"]-infos.loc["Nbe NaN"])\
                         .T.rename(index = {0:"Nbe Entrées"}))
    infos = infos.append(pd.DataFrame(df.isna().sum()/df.shape[0]*100).T.rename(index={0:'% NaN'}))
    infos = infos.append(pd.DataFrame([df[col].unique().size for col in df.columns],index=df.columns)\
                               .T.rename(index={0:'Nbe uniques'}))
    return infos

In [ ]:
###############
#     Data    #
###############
infos_df(data)

In [ ]:
###############
#   Country   #
###############
infos_df(country)

In [ ]:
#################
#     Series    #
#################
infos_df(series)

In [ ]:
#########################
#     Country-Series    #
#########################
infos_df(cnt_ser)

In [ ]:
#################
#    FootNote   #
#################
infos_df(footnote)

## 2. Vérification de la qualité des données
### 2.1 Données dupliquées ou contradictoires
#### Noms et codes des pays

- Les tables "Data" et "Country" n'ont pas le même nombre d'entrée uniques pour la colonne "Country Code" : la table "Country" contient un pays en moins. On cherche à déterminer ce pays :

In [ ]:
# Fonction qui trouve les éléments différents dans deux tableaux (réciproque)

def Diff(tab1, tab2):  # le nombre total de différences
    tab_dif = [i for i in tab1 + tab2 if i not in tab1 or i not in tab2] 
    return tab_dif 

def Diff_bis(tab1, tab2): # compare en détaillant les entrées différentes de tab1, puis de tab2
    tab_dif = [i for i in tab1 + tab2 if i not in tab1 or i not in tab2] 
    return (set(tab1)-set(tab2),set(tab2)-set(tab1))

In [ ]:
pays = Diff(list(data["Country Code"].unique()),list(country["Country Code"].unique()))
print("Code et nom du pays manquant dans Country : {},{}"\
  .format(pays,[data["Country Name"][data["Country Code"]==pays[i]].iloc[0] for i in range(len(pays))]))

- On souhaite ensuie tester la correspondance bijective entre plusieurs paires de colonnes d'une même table afin de pouvoir répondre à des questions du type :

> __Une entrée de 'nom_col1' correspond-elle à une et une seule entrée de la colonne 'nom_col2' dans la table 'df' ?__

On crée une fonction pour cela :

In [ ]:
# Fonction comparant la correspondance unique entre les valeurs d'une même ligne de deux colonnes d'une base
# (bijection entre les valeurs de col1 et de col2)

def Adeq (df, nom_col1,nom_col2):
    mon_zip = zip(df[nom_col1], df[nom_col2]) # associe les entrées des deux colonnes en tuples
    nbe_comb = len(set(mon_zip)) # retourne les valeurs uniques des tuples
    return nbe_comb==df[nom_col1].unique().size # si le nbe est le même que les valeurs uniques, c'est bon

# Version très lourde de la même fonction
# def Adeq_Col(df, nom_col1,nom_col2): # dans une même base df
#     tab = []
#     for i in range(len(df[nom_col1].unique())):
#         uni_i = df[nom_col1].unique()[i]
#         # liste des valeurs de col2 correspondant à la ième valeur unique de col1
#         li = df[df[nom_col1]==uni_i][nom_col2]
#         temp = 0
#         for j in range(len(li)): # comparaison de toutes les valeurs de li une à une
#             if li.iloc[j] != li.iloc[0]: # on prend la première valeur (0) comme référence
#                 temp+=1
#         tab.append(temp)
#     return temp # la valeur retournée est le nombre de valeurs différentes

Puis on teste les couples de colonnes listés ci-dessous :
- Table "Data" : "Country Code", "Country Name"
- Table "Data" : "Indicator Code", "Indicator Name"
- Table "Country" : "Country Code", "Short Name"
- Table "Country" : "Country Code", "Table Name"
- Table "Country" : "Country Code", "Long Name"
- Table "Series" : "Series Code", "Indicator Name"

In [ ]:
print("Data : {} {}"\
      .format(Adeq(data, "Country Code", "Country Name"), Adeq(data, "Indicator Code", "Indicator Name")))
print("Country : {} {} {}"\
      .format(Adeq(country, "Country Code", "Short Name"), Adeq(country, "Country Code", "Table Name"),\
        Adeq(country, "Country Code", "Long Name")))
print("Series : {}".format(Adeq(series, "Series Code", "Indicator Name")))

In [ ]:
a = [3,2,1,5,1,6,1]
b = ['c','b','a','e','a','u','a']
z = zip(a,b)
set(z)

Y a-t-il la même correspondance entre les codes pays ("Country Code") et les noms de pays dans les tables "Data" ("Country Name") et "Country" ("Table Name") ?
Si oui, on ne gardera qu'une des deux colonnes.

In [ ]:
#set(zip(sorted(data["Country Name"].unique()), sorted(country["Table Name"].unique())))
data_gb_cnt = data.groupby("Country Code").count() #"Country Name"

##### Noms et codes des indicateurs statistiques

In [ ]:
Il est

On remarque que malgré les noms différents, les colonnes "Indicator Code" de la table "Data" et "Series Code" de la table "Series" semblent faire référence au même code d'indicateur statistique, et ont, elles aussi, le même nombre d'entrées uniques de 3665.
Regardons si les entrées uniques de ces deux colonnes correspondent :

In [ ]:
code = Diff_bis(list(data["Indicator Code"].unique()),list(series["Series Code"].unique()))
len(code[0]),len(code[1])

53 entrées uniques de la colonne "Indicator Code" de la table "Data" ne sont pas dans la table "Series" et que le même nombre 53 d'entrées uniques de la colonne "Series Code" ne sont pas dans la table "Data".

In [ ]:
list(code[0])[:2], list(code[1])[:2]

On constate que tous les codes différents de la colonne "Series Code" (table "Series") ont certaines lettres en minuscule.
Essayons donc de mettre toutes les entrées en majuscule et de refaire la comparaison.

In [ ]:
# pour la table "Series"
test_series = pd.Series([series["Series Code"][i].upper() for i in range(series["Series Code"].index.size)],\
                 index = series["Series Code"].index)
# pour la table "Data"
test_data = pd.Series([data["Indicator Code"][i].upper() for i in range(data["Indicator Code"].index.size)],\
                 index = data["Indicator Code"].index)

code = Diff_bis(list(test_data.unique()),list(test_series.unique()))
len(code[0]),len(code[1])

In [ ]:
list(code[0]), list(code[1])

La liste ci-dessus montre qu'il faudra également retirer les espaces de la colonne "Series Code" de la table "Series".

In [ ]:
test_series = pd.Series([test_series.iloc[i].replace(" ", "") for i in range(test_series.index.size)],\
                 index = series["Series Code"].index)

In [ ]:
code = Diff_bis(list(test_data.unique()),list(test_series.unique()))
len(code[0]),len(code[1])

On remarque par ailleurs que les colonnes "Indicator Name" des tables "Data" et "Series" ont le même nombre d'entrées uniques (3665). Vérifions si ces entrées uniques correspondent :

In [ ]:
indic = Diff_bis(list(data["Indicator Name"].unique()),list(series["Indicator Name"].unique()))
len(indic[0]), len(indic[1])

On remarque que 462 entrées uniques de la colonne "Indicator Name" de la table "data" ne sont pas dans la table "series" et que le même nombre 462 d'entrées uniques de la même colonne de la table "series" ne sont pas dans la table "data".

Cela pourrait-il indiquer qu'il s'agirait des mêmes entrées qui ne présentent que de petites variations d'écriture, comme c'est le cas pour les deux entrées ci-dessous ?

In [ ]:
indic = Diff_bis(list(data["Indicator Code"].unique()),list(series["Series Code"].unique()))
len(indic[0]), len(indic[1])

In [ ]:
list(indic[0])[7],list(indic[1])[8]

In [ ]:
a = data["Indicator Code"][data["Indicator Name"]=='SABER: (School Autonomy Accountability) Policy Goal 2: Level of autonomy in personnel management'].iloc[0:3]
b = series["Series Code"][series["Indicator Name"]=='SABER: (School Autonomy and Accountability) Policy Goal 2: Level of autonomy in personnel management']
a,b

#### Colonnes de la table "CountrySeries" et "FootNote"

Les entrées des colonnes "CountryCode" et "SeriesCode" des tables "CountrySeries" et "FootNote" existent-ils dans la liste de référence "Country Code" de la table "Data" ?

Les années de la table "FootNote" sont-elles dans les années correspondant aux noms de colonnes de la table "Data" ?

In [ ]:
# liste des années de la colonne "Year" de la table "FootNote"
#set(footnote["Year"].unique())
footnote[footnote["Year"]=='yr2012']

## Exploration de la table "Data"

Les indicateurs qui nous intéressent sont ceux des dernières années.
On cherche à savoir :
- combien d'indicateurs environ sont disponibles dans les dernières années
- quels sont les pays qui ont le plus d'indicateurs disponibles dans les dernières années
- quels sont les indicateurs le plus souvent disponible 

In [ ]:
# Tableau des nombres d'indicateurs dispo pour chaque pays et chaque année 
nb_ind_cnt = data.groupby(['Country Name', 'Country Code']).count()

In [ ]:
# Pays ayant un nbe d'indicateur non nul en 2017
nb_ind_cnt[nb_ind_cnt['2017']!=0]

In [ ]:
# Nbe de pays ayant un nbe d'indicateur supérieur à n en fonction des années

def calc_nb_pay_rens (tab_n):
    tab = [ [nb_ind_cnt[nb_ind_cnt[str(i)]>j].index.size for i in years] for j in tab_n ]
    res = np.array(tab).T
    return res

countrys = [nb_ind_cnt.index[i][0] + " - " + nb_ind_cnt.index[i][1] for i in range(nb_ind_cnt.index.size)]
years = list(range(1970,2018))+list(range(2020,2105,5))
tab_n = [0,2,10,50,100, 200, 500]
x = years
y = calc_nb_pay_rens(tab_n)
len(y[:, 1]), len(x)

In [ ]:
fig = plt.figure(figsize = (18,6))

colors = ["#bd5db0","#70a845","#727bcc","#b49242","#cc566c","#4aad92","#ca6037"]
labels = ["i="+str(i) for i in tab_n]

plot1 = plt.subplot(1,2,1)
[plot1.plot(x, y[:,i], '-o', label = labels[i], color = colors[i]) for i in range(len(tab_n))]
plot1.set_ylim(0,250)
plot1.legend(loc = 'lower right')
plot1.set_xlabel("années", fontsize = 14)
plot1.set_ylabel("nbe de pays", fontsize = 14)
plot1.set_title("Pays ayant plus de i indicateurs", fontsize = 18, fontweight = 'bold')

plot2 = plt.subplot(1,2,2)
[plot2.plot(x, y[:,i], '-o', label = labels[i], color = colors[i]) for i in range(len(tab_n))]
plot2.set_xlim(1980,2018)
plot2.set_ylim(75,250)
plot2.legend(loc = 'lower right')
plot2.set_xlabel("années", fontsize = 14)
plot2.set_ylabel("nbe de pays", fontsize = 14)
plot2.set_title("Pays ayant plus de i indicateurs (zoom)", fontsize = 18, fontweight = 'bold')
plt.show() 

In [ ]:
# Classement des pays en fonction du nombre d'indicateurs disponibles en 2017
fig = plt.figure(figsize = (18,6))
x = countrys
y = nb_ind_cnt['2017']
plt.plot(x, y, '-o')
plt.
plt.show() 

In [ ]:
arrays = [[1, 1, 2, 2], ['red', 'blue', 'red', 'blue']]
pd.MultiIndex.from_arrays(arrays, names=('number', 'color'))
# MultiIndex(levels=[[1, 2], ['blue', 'red']],
#            codes=[[0, 0, 1, 1], [1, 0, 1, 0]],
#            names=['number', 'color'])

In [ ]:
# Nbe d'indicateurs (sur 3665 = max) dispo par année pour un pays

tab = [data_gb_cnt[data_gb_cnt.index=='FRA'][year].values for year in data_gb_cnt.columns[3:].values]
fig = plt.figure(figsize = (18,6))
x = years
y = tab
plt.plot(x, y, '-o')
plt.show() 

## Exploration des mots clés dans les noms d'indicateurs

In [ ]:
def contAny (cars, mot):
    return any([True if c in cars else False for c in mot])

def contAll (cars, mot):
    return all([True if c in cars else False for c in mot])

def enum_mots_cmpt(li_phrases, nb):
    li_mots = " ".join(li_phrases).split(" ")
    li_mots_net = sorted([mot for mot in li_mots if (mot != '') and not contAll('-)%.(,', mot)])
    cpt = Counter(li_mots_net)
    return cpt.most_common(nb)

# Function that control the color of the words
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# WARNING: the scope of variables is used to get the value of the "tone" variable
# I could not find the way to pass it as a parameter of "random_color_func()"
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
def random_color_func(word=None, font_size=None, position=None, orientation=None, font_path=None, random_state=None):
    h = int(360.0 * tone / 255.0)
    s = int(100.0 * 255.0 / 255.0)
    l = int(100.0 * float(np.random.randint(70, 120) / 255.0))
    return "hsl({}, {}%, {}%)".format(h, s, l)
tone = 10.0 # define the color of the words

In [ ]:
tab_indic = data["Indicator Name"]
enum_mots_cmpt(tab_indic, 5)
# I define the dictionary used to produce the wordcloud
words = dict()
list_pop = ["in", "of, ""a", "and", "Per", "by", "the", "with", "to", "from",\
            "for", "who", "on", "are"]
dict_key = words.keys()
[words.pop(w) for w in list_pop if w in dict_key]

In [ ]:
occurences = enum_mots_cmpt(tab_indic, 100)
for s in occurences:
    words[s[0]] = s[1]

In [ ]:
# UPPER PANEL: WORDCLOUD
fig = plt.figure(1, figsize=(18,13))
ax1 = fig.add_subplot(2,1,1)
wordcloud = WordCloud(width=1000,height=300, background_color='black', max_words=1628,relative_scaling=1,\
                      color_func = random_color_func, normalize_plurals=False)
wordcloud.generate_from_frequencies(words)
ax1.imshow(wordcloud, interpolation="bilinear")
ax1.axis('off')
#_____________________________________________
# LOWER PANEL: HISTOGRAMS
ax2 = fig.add_subplot(2,1,2)
y_axis = [i[1] for i in occurences]
x_axis = [k for k,i in enumerate(occurences)]
x_label = [i[0] for i in occurences]
plt.xticks(rotation=85, fontsize = 15)
plt.yticks(fontsize = 15)
plt.xticks(x_axis, x_label)
plt.ylabel("Nb. of occurences", fontsize = 18, labelpad = 10)
ax2.bar(x_axis, y_axis, align = 'center', color='g')
#_______________________
plt.title("Keywords popularity",bbox={'facecolor':'k', 'pad':5},color='w',fontsize = 25)
plt.show()

## Détection des outliers

In [ ]:
# prend un tableau de valeurs, calcule la moyenne, l'écart type
# et renvoie les valeurs au-delà ou en-deça de x fois l'écart type
def detOutliers(df,col,x):
    moy = df[col].mean()
    std = df[col].std()
    out_val = [nb for nb in df[col].values if ((nb<moy-(x*std)) or (nb>moy+(x*std)))]
    return df[df[col].isin(out_val)]

my_tab = [1,3,0,2,5]
df = pd.DataFrame({'my_col' : my_tab})

tab_df = [detOutliers(df, "my_col",x) for x in np.linspace(0,2.5,7)]
[plt.plot(df.index, df["my_col"].values,'o-', label = "x = "+str(df.index.values)) for df in tab_df]
plt.legend()
plt.show()

## -------------- BROUILLONS -------------

In [ ]:
tab1 = list(['a', 'b', 'c', 'd', 'e'])
tab2 = list(['adfc', 'c', 'd', 'e'])

list1 = [10, 15, 20, 45, 30, 35, 25]
list2 = [25, 40, 35] 


def Diff(tab1, tab2): 
    tab_dif = [i for i in tab1 + tab2 if i not in tab1 or i not in tab2] 
    return tab_dif 

Diff(list1, list2)
(set(tab2)-set(tab1)),(set(tab1)-set(tab2))


In [ ]:
## .str.match (s'applique à un index)

df = pd.DataFrame([0,1,2,3,4], index = ['a', 'b', 'Unnamed', 'd', 'e'])
df = df.T
~df.columns.str.match('Unnamed')
df.loc[:, [True, False, True]]

In [ ]:
## groupby, merge, join, concat etc.

#DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False,
# right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo'], 'value': [1, 2, 3, 1], 'value2': [100, 200, 300, 500]})
df2 = pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo'], 'value': [5, 6, 7, 8]})
#df1.merge(df2, right_on='rkey')
df1.merge(df2, left_on = 'lkey', right_on='rkey')
df1.groupby(['value', 'lkey']).count()